# Elgato OBS

In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
# iterate over the dataframe 'Data' and calculate angles
import numpy as np


def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(
        a[1] - b[1], a[0] - b[0]
    )
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle


def getLandmarkXYZ(coordinates, landmark):
    for coord in coordinates:
        if coord["Landmark"] == landmark:
            return [coord["X"], coord["Y"]]


def calculateAngles(coords, joints):
    angles = []
    for joint in joints:
        landmark_first = getLandmarkXYZ(coords, joint[0])
        landmark_mid = getLandmarkXYZ(coords, joint[1])
        landmark_end = getLandmarkXYZ(coords, joint[2])

        angle = calculate_angle(landmark_first, landmark_mid, landmark_end)
        angles.append(angle)

    return angles


all_angles = [
    (14, 12, 24),
    (13, 11, 23),
    (16, 14, 12),
    (15, 13, 11),
    (12, 24, 26),
    (11, 23, 25),
    (24, 26, 28),
    (23, 25, 27),
    (11, 12, 24),
    (12, 11, 23),
    (26, 24, 23),
    (25, 23, 24),
]
angle_columns_names = [
    "left_arm",
    "right_arm",
    "left_elbow",
    "right_elbow",
    "left_waist_leg",
    "right_waist_leg",
    "left_knee",
    "right_knee",
    "leftup_chest_inside",
    "rightup_chest_inside",
    "leftlow_chest_inside",
    "rightlow_chest_inside",
]
len(all_angles), len(angle_columns_names)

(12, 12)

In [3]:
# VIDEO FEED CHECK
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow("Video Feed", frame)

    if cv2.waitKey(10) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [4]:
def draw_animation(image, line_start, angle, angle_name):
    line_length = 50

    # Draw the first line in red
    line_end_1 = (int(line_start[0] + line_length), line_start[1])
    cv2.line(image, line_start, line_end_1, (0, 0, 255), 2)

    # Draw the second line at an angle in blue
    line_end_2 = (
        int(line_end_1[0] + line_length * np.cos(np.radians(angle))),
        int(line_end_1[1] - line_length * np.sin(np.radians(angle))),
    )
    cv2.line(image, line_end_1, line_end_2, (255, 0, 0), 2)

    # Display angle and name text
    angle_text = f"{angle_name}:{(angle):.2f}deg"
    cv2.putText(
        image,
        angle_text,
        (line_start[0] - 40, line_start[1] + 20),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.4,
        (0, 0, 0),
        1,
    )


def processPoseLandmark(results):
    # Draw landmarks on the frame

    index = 0
    keypoints = []
    for data_point in results:
        thisXYZ = {
            "X": data_point.x,
            "Y": data_point.y,
            "Z": data_point.z,
            "Visibility": data_point.visibility,
            "Landmark": index,
        }
        keypoints.append(thisXYZ)
        index += 1

    return keypoints

In [5]:
# INITIATE ANGLE WINDOW
landscape_image = np.ones((500, 850, 3), dtype=np.uint8) * 255

# Define the initial starting points for each animation in a 4x3 grid
start_points_grid = [(50 + (i % 4) * 200, 50 + (i // 4) * 150) for i in range(12)]

# Extract relevant columns from the DataFrame
angle_columns = [
    "left_arm",
    "right_arm",
    "left_elbow",
    "right_elbow",
    "left_waist_leg",
    "right_waist_leg",
    "left_knee",
    "right_kneee",
    "leftup_chest_inside",
    "rightup_chest_inside",
    "leftlow_chest_inside",
    "rightlow_chest_inside",
    "leg_spread",
]


# OPEN CV - VIDEO FEED
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass

        # Render detections
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 0, 30), thickness=2, circle_radius=1),
        )

        cv2.imshow("gym-motion-pose-ai v1.0", image)

        # calculate angles on a separate window
        json = processPoseLandmark(landmarks)
        angles = calculateAngles(json, all_angles)
        landscape_image = np.ones((500, 850, 3), dtype=np.uint8) * 255

        for i, (start_point, angle_column) in enumerate(
            zip(start_points_grid, angle_columns)
        ):
            angle_value = angles[i]
            draw_animation(
                landscape_image, start_point, 180.0 - angle_value, angle_column
            )

        cv2.imshow("Grid of Animations", landscape_image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

NameError: name 'landmarks' is not defined